In [ ]:
# -*- coding: utf-8 -*-
"""
Алгоритм покрытия поля с учетом разворотных полос и радиусов поворота трактора.
Реализует зигзагообразное покрытие (Boustrophedon) с плавными разворотами.
"""

import ee
import geemap
import geopandas as gpd
import pandas as pd
import shapely.geometry as geom
from shapely.ops import linemerge, polygonize, unary_union
from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString
import numpy as np
import math
from scipy.spatial import KDTree
import json
import folium
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Инициализация GEE
print("=== ИНИЦИАЛИЗАЦИЯ GOOGLE EARTH ENGINE ===")
try:
    ee.Initialize(project='ee-pochyomin99')
    print("GEE успешно инициализирован")
except Exception as e:
    print(f"Ошибка инициализации GEE: {e}")
    try:
        ee.Authenticate()
        ee.Initialize(project='ee-pochyomin99')
    except:
        print("Требуется аутентификация: earthengine authenticate")
        sys.exit(1)

# ПАРАМЕТРЫ СИСТЕМЫ
params = {
    # Параметры агрегата
    'working_width': 5.04,            # Ширина захвата плуга, м
    'hitch_length': 2.0,              # Длина сцепки, м
    'min_turn_radius': 7.2,           # Минимальный радиус поворота К-700, м
    'tool_length': 21.1,              # Длина орудия, м
    'implement_width': 3.0,           # Ширина орудия в транспортном положении, м
    
    # Параметры разворота
    'headland_width': 15.0,           # Ширина разворотной полосы по краю поля (мин: 2*радиус)
    'turn_type': 'fishbone',          # Тип разворота: 'fishbone', 'half-circle', 'teardrop'
    'allow_exit_field': False,        # Разрешить выезд за границы для разворота
    'smoothing_factor': 0.5,          # Сглаживание траектории (0-1)
    
    # Параметры покрытия
    'coverage_strategy': 'boustrophedon',  # 'boustrophedon', 'spiral', 'contour'
    'min_swath_length': 20.0,         # Минимальная длина гона, м
    'edge_buffer': 1.0,               # Отступ от края внутри разворотной полосы, м
    
    # Пути к файлам
    'field_path': r"E:\Gis_evening\FieldRoad\Pole.shp",
    'entry_exit_path': r"E:\Gis_evening\FieldRoad\In_out.shp",
    'output_track_path': r"E:\Gis_evening\FieldRoad\optimized_track.geojson",
    'output_map_path': r"E:\Gis_evening\FieldRoad\optimized_track_map.html",
    'output_headland_path': r"E:\Gis_evening\FieldRoad\headland.geojson",
    'output_swaths_path': r"E:\Gis_evening\FieldRoad\swaths.geojson",
    
    # Оптимизация
    'population_size': 40,
    'max_generations': 80,
    'encoding': 'utf-8',
}

# ФУНКЦИЯ ДЛЯ СОЗДАНИЯ ВНУТРЕННЕЙ ГРАНИЦЫ (разворотная полоса)
def create_headland_boundary(field_polygon, headland_width):
    """
    Создает внутреннюю границу поля для разворотов.
    Гоны будут обрабатываться только внутри этой границы.
    """
    print(f"  Создание разворотной полосы шириной {headland_width} м...")
    
    # Создаем внутренний отступ (разворотную полосу)
    headland_polygon = field_polygon.buffer(-headland_width)
    
    # Проверяем, что отступ не уничтожил полигон
    if headland_polygon.is_empty:
        print(f"  Внимание: разворотная полоса слишком широкая!")
        # Уменьшаем ширину до половины и пробуем снова
        headland_width = headland_width / 2
        headland_polygon = field_polygon.buffer(-headland_width)
        
        if headland_polygon.is_empty:
            print(f"  Ошибка: поле слишком маленькое для разворотной полосы")
            return field_polygon, None
    
    # Создаем полигон разворотной полосы (разница между исходным и внутренним)
    headland_zone = field_polygon.difference(headland_polygon)
    
    # Рассчитываем площадь
    field_area = field_polygon.area
    headland_area = headland_zone.area
    work_area = headland_polygon.area
    
    print(f"    Площадь поля: {field_area:.0f} м²")
    print(f"    Площадь рабочей зоны: {work_area:.0f} м² ({work_area/field_area*100:.1f}%)")
    print(f"    Площадь разворотной полосы: {headland_area:.0f} м² ({headland_area/field_area*100:.1f}%)")
    
    return headland_polygon, headland_zone

# ФУНКЦИЯ ДЛЯ РАСЧЕТА НАПРАВЛЕНИЯ ОБРАБОТКИ (оптимизированная)
def calculate_processing_direction_optimized(field_gdf):
    """Определяет оптимальное направление обработки с использованием GEE."""
    print("  Определение направления обработки...")
    
    try:
        # Преобразуем в WGS84 для GEE
        wgs84_gdf = field_gdf.to_crs('EPSG:4326')
        poly = wgs84_gdf.geometry.iloc[0]
        
        # Упрощаем полигон для GEE
        if poly.geom_type == 'MultiPolygon':
            poly = poly.geoms[0]
        
        # Упрощаем, сохраняя топологию
        simple_poly = poly.simplify(0.001, preserve_topology=True)
        coords = list(simple_poly.exterior.coords)
        
        # GEE ожидает координаты в формате [долгота, широта]
        ee_coords = [[coord[0], coord[1]] for coord in coords]
        field_ee_geom = ee.Geometry.Polygon(ee_coords)
        
        # Используем NASADEM для рельефа
        dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
        terrain = ee.Terrain.products(dem)
        aspect = terrain.select('aspect')
        
        # Вычисляем средний аспект
        mean_aspect_dict = aspect.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=field_ee_geom,
            scale=100,  # Разрешение 100м для больших полей
            maxPixels=1e9
        )
        
        mean_aspect = mean_aspect_dict.get('aspect').getInfo()
        
        if mean_aspect is None:
            print("  Не удалось определить направление склона, используем 0°")
            return 0.0
        else:
            # Направление обработки - перпендикулярно склону
            processing_direction = (mean_aspect + 90) % 360
            print(f"    Среднее направление склона: {mean_aspect:.1f}°")
            print(f"    Направление обработки: {processing_direction:.1f}°")
            return processing_direction
            
    except Exception as e:
        print(f"  Ошибка GEE: {e}")
        print("  Используем направление 0° (с севера на юг)")
        return 0.0

# ФУНКЦИЯ ГЕНЕРАЦИИ ГОНОВ С УЧЕТОМ РАЗВОРОТНОЙ ПОЛОСЫ
def generate_swaths_with_headland(work_polygon, processing_angle, swath_width, min_swath_length):
    """
    Генерирует гоны внутри рабочей зоны (за вычетом разворотной полосы).
    """
    print("  Генерация гонов внутри рабочей зоны...")
    
    # Преобразуем угол в радианы
    angle_rad = math.radians(processing_angle)
    dir_x, dir_y = math.cos(angle_rad), math.sin(angle_rad)
    perp_x, perp_y = -dir_y, dir_x  # Перпендикулярное направление
    
    # Получаем границы рабочей зоны
    bounds = work_polygon.bounds
    minx, miny, maxx, maxy = bounds
    
    # Центр рабочей зоны
    center_x, center_y = (minx + maxx) / 2, (miny + maxy) / 2
    
    # Собираем все вершины рабочей зоны для определения диапазона
    vertices = []
    if work_polygon.geom_type == 'MultiPolygon':
        for poly in work_polygon.geoms:
            vertices.extend(list(poly.exterior.coords))
            for interior in poly.interiors:
                vertices.extend(list(interior.coords))
    else:
        vertices = list(work_polygon.exterior.coords)
        for interior in work_polygon.interiors:
            vertices.extend(list(interior.coords))
    
    # Проекции вершин на перпендикулярное направление
    projections = [vx * perp_x + vy * perp_y for vx, vy in vertices]
    min_proj, max_proj = min(projections), max(projections)
    
    # Добавляем небольшой отступ для полного покрытия
    buffer_dist = swath_width * 0.1
    start_proj = min_proj - buffer_dist
    end_proj = max_proj + buffer_dist
    
    # Количество гонов
    num_swaths = int(math.ceil((end_proj - start_proj) / swath_width))
    print(f"    Теоретическое количество гонов: {num_swaths}")
    
    # Длина линии должна гарантированно пересекать рабочую зону
    diagonal = math.sqrt((maxx - minx)**2 + (maxy - miny)**2)
    line_length = diagonal * 1.5
    
    swaths = []
    
    # Генерируем гоны
    for i in range(num_swaths + 1):
        # Положение линии
        line_proj = start_proj + i * swath_width
        
        # Координаты центра линии
        center_proj_x = center_x + (line_proj - (min_proj + max_proj)/2) * perp_x
        center_proj_y = center_y + (line_proj - (min_proj + max_proj)/2) * perp_y
        
        # Создаем длинную линию
        long_line = LineString([
            (center_proj_x - dir_x * line_length, center_proj_y - dir_y * line_length),
            (center_proj_x + dir_x * line_length, center_proj_y + dir_y * line_length)
        ])
        
        # Обрезаем линию рабочей зоной
        clipped = long_line.intersection(work_polygon)
        
        if clipped.is_empty:
            continue
            
        # Обрабатываем разные типы геометрий
        if clipped.geom_type == 'MultiLineString':
            segments = list(clipped.geoms)
            # Сортируем сегменты по положению
            segments.sort(key=lambda s: s.coords[0][0] * dir_x + s.coords[0][1] * dir_y)
            
            for segment in segments:
                if segment.length >= min_swath_length:
                    swaths.append(segment)
                    
        elif clipped.geom_type == 'LineString':
            if clipped.length >= min_swath_length:
                swaths.append(clipped)
    
    # Объединяем короткие соседние сегменты
    merged_swaths = []
    used = [False] * len(swaths)
    
    # Параметры объединения
    max_gap = swath_width * 0.8  # Максимальный разрыв для объединения
    angle_tolerance = math.radians(5)  # Допуск по углу
    
    for i in range(len(swaths)):
        if used[i]:
            continue
            
        current = swaths[i]
        current_coords = list(current.coords)
        
        # Проверяем направление текущего сегмента
        if len(current_coords) >= 2:
            dx1 = current_coords[-1][0] - current_coords[0][0]
            dy1 = current_coords[-1][1] - current_coords[0][1]
            current_angle = math.atan2(dy1, dx1) if abs(dx1) > 0.001 else math.pi/2
            
            # Ищем соседние сегменты для объединения
            for j in range(i + 1, len(swaths)):
                if used[j]:
                    continue
                    
                other = swaths[j]
                other_coords = list(other.coords)
                
                if len(other_coords) < 2:
                    continue
                
                # Проверяем направление другого сегмента
                dx2 = other_coords[-1][0] - other_coords[0][0]
                dy2 = other_coords[-1][1] - other_coords[0][1]
                other_angle = math.atan2(dy2, dx2) if abs(dx2) > 0.001 else math.pi/2
                
                # Проверяем коллинеарность
                angle_diff = abs(current_angle - other_angle)
                if angle_diff > math.pi:
                    angle_diff = 2 * math.pi - angle_diff
                
                if angle_diff > angle_tolerance:
                    continue  # Сегменты не коллинеарны
                
                # Проверяем расстояние между концами
                dist_end_to_start = Point(current_coords[-1]).distance(Point(other_coords[0]))
                dist_start_to_end = Point(current_coords[0]).distance(Point(other_coords[-1]))
                dist_end_to_end = Point(current_coords[-1]).distance(Point(other_coords[-1]))
                dist_start_to_start = Point(current_coords[0]).distance(Point(other_coords[0]))
                
                min_dist = min(dist_end_to_start, dist_start_to_end, 
                              dist_end_to_end, dist_start_to_start)
                
                if min_dist <= max_gap:
                    # Объединяем сегменты
                    if min_dist == dist_end_to_start:
                        new_coords = current_coords + other_coords
                    elif min_dist == dist_start_to_end:
                        new_coords = other_coords + current_coords
                    elif min_dist == dist_end_to_end:
                        new_coords = current_coords + list(reversed(other_coords))
                    else:  # start_to_start
                        new_coords = list(reversed(other_coords)) + current_coords
                    
                    try:
                        current = LineString(new_coords)
                        current_coords = list(current.coords)
                        used[j] = True
                    except:
                        pass
        
        merged_swaths.append(current)
        used[i] = True
    
    # Фильтруем слишком короткие гоны
    final_swaths = [s for s in merged_swaths if s.length >= min_swath_length]
    
    # Сортируем гоны по положению
    final_swaths.sort(key=lambda s: 
        s.centroid.x * perp_x + s.centroid.y * perp_y)
    
    print(f"    Сгенерировано гонов: {len(final_swaths)}")
    
    # Оценка покрытия
    if len(final_swaths) > 0:
        total_length = sum(s.length for s in final_swaths)
        coverage_area = total_length * swath_width
        work_area = work_polygon.area
        
        if work_area > 0:
            coverage_percent = (coverage_area / work_area) * 100
            print(f"    Ожидаемое покрытие рабочей зоны: {coverage_percent:.1f}%")
            print(f"    Общая длина гонов: {total_length:.1f} м")
    
    return final_swaths

# ФУНКЦИЯ ДЛЯ СОЗДАНИЯ ПЛАВНОГО РАЗВОРОТА
def create_smooth_turn(start_point, start_angle, end_point, end_angle, turn_radius, turn_type='fishbone'):
    """
    Создает плавный разворот между двумя точками с заданными углами.
    Возвращает список сегментов траектории разворота.
    """
    segments = []
    
    # Преобразуем углы в радианы
    start_angle_rad = math.radians(start_angle)
    end_angle_rad = math.radians(end_angle)
    
    # Векторы направлений
    start_dir = Point(math.cos(start_angle_rad), math.sin(start_angle_rad))
    end_dir = Point(math.cos(end_angle_rad), math.sin(end_angle_rad))
    
    # Координаты начальной и конечной точек
    x1, y1 = start_point.x, start_point.y
    x2, y2 = end_point.x, end_point.y
    
    if turn_type == 'half-circle':
        # Полукруговой разворот (простейший)
        # Вычисляем центр дуги
        angle_diff = end_angle_rad - start_angle_rad
        if abs(angle_diff) > math.pi:
            angle_diff = angle_diff - 2*math.pi if angle_diff > 0 else angle_diff + 2*math.pi
        
        # Создаем дугу
        if abs(angle_diff) > 0.1:  # Если нужно повернуть
            # Центр поворота
            turn_center_x = x1 + turn_radius * math.cos(start_angle_rad + math.pi/2)
            turn_center_y = y1 + turn_radius * math.sin(start_angle_rad + math.pi/2)
            
            # Генерируем точки дуги
            num_points = 20
            arc_points = []
            for i in range(num_points + 1):
                t = i / num_points
                angle = start_angle_rad + angle_diff * t
                px = turn_center_x + turn_radius * math.cos(angle + math.pi/2)
                py = turn_center_y + turn_radius * math.sin(angle + math.pi/2)
                arc_points.append((px, py))
            
            segments.append(LineString(arc_points))
    
    elif turn_type == 'fishbone':
        # Разворот "рыбьей костью" (зигзагообразный)
        # Простой вариант: два прямых отрезка с плавным переходом
        
        # Вычисляем промежуточную точку
        mid_x = (x1 + x2) / 2
        mid_y = (y1 + y2) / 2
        
        # Добавляем смещение для плавности
        offset = turn_radius * 0.5
        
        # Первый отрезок (от начальной точки)
        p1 = Point(x1 + offset * math.cos(start_angle_rad), 
                  y1 + offset * math.sin(start_angle_rad))
        
        # Второй отрезок (к конечной точке)
        p2 = Point(x2 - offset * math.cos(end_angle_rad), 
                  y2 - offset * math.sin(end_angle_rad))
        
        segments.append(LineString([start_point, p1]))
        segments.append(LineString([p1, p2]))
        segments.append(LineString([p2, end_point]))
    
    else:  # Прямой переход (по умолчанию)
        segments.append(LineString([start_point, end_point]))
    
    return segments

# ФУНКЦИЯ ОПТИМИЗАЦИИ ПОРЯДКА ГОНОВ
def optimize_swath_order(swaths, entry_point, exit_point, processing_angle):
    """
    Оптимизирует порядок прохода гонов для минимизации длины пути.
    Использует жадный алгоритм с элементами случайного поиска.
    """
    print("  Оптимизация порядка гонов...")
    
    if len(swaths) < 2:
        return list(range(len(swaths))), swaths
    
    # Подготавливаем данные о гонах
    swath_data = []
    for i, swath in enumerate(swaths):
        coords = list(swath.coords)
        start_point = Point(coords[0])
        end_point = Point(coords[-1])
        
        # Определяем направление гона (от начала к концу)
        dx = end_point.x - start_point.x
        dy = end_point.y - start_point.y
        swath_angle = math.degrees(math.atan2(dy, dx))
        
        # Центр гона для сортировки
        center = swath.centroid
        
        swath_data.append({
            'id': i,
            'line': swath,
            'start': start_point,
            'end': end_point,
            'length': swath.length,
            'angle': swath_angle,
            'center': center,
            'x_proj': center.x * math.cos(math.radians(processing_angle)) + 
                     center.y * math.sin(math.radians(processing_angle))
        })
    
    # Сортируем гоны по проекции на направление обработки
    swath_data.sort(key=lambda x: x['x_proj'])
    
    # Простой жадный алгоритм
    current_point = entry_point
    visited = [False] * len(swath_data)
    order = []
    
    # Функция для вычисления расстояния до гона
    def distance_to_swath(point, swath):
        to_start = point.distance(swath['start'])
        to_end = point.distance(swath['end'])
        return min(to_start, to_end), to_start <= to_end
    
    # Основной цикл жадного алгоритма
    while len(order) < len(swath_data):
        best_dist = float('inf')
        best_idx = -1
        best_from_start = True
        
        # Ищем ближайший непосещенный гон
        for i, swath in enumerate(swath_data):
            if visited[i]:
                continue
            
            dist, from_start = distance_to_swath(current_point, swath)
            if dist < best_dist:
                best_dist = dist
                best_idx = i
                best_from_start = from_start
        
        if best_idx == -1:
            break
        
        # Добавляем гон в порядок
        order.append((best_idx, best_from_start))
        visited[best_idx] = True
        
        # Обновляем текущую позицию
        if best_from_start:
            current_point = swath_data[best_idx]['end']
        else:
            current_point = swath_data[best_idx]['start']
    
    # Улучшаем решение случайными перестановками
    best_order = order.copy()
    best_length = calculate_path_length(best_order, swath_data, entry_point, exit_point)
    
    # Локальный поиск с случайными перестановками
    for iteration in range(100):
        # Случайная перестановка двух соседних гонов
        if len(order) >= 2:
            i = np.random.randint(0, len(order) - 1)
            new_order = order.copy()
            new_order[i], new_order[i + 1] = new_order[i + 1], new_order[i]
            
            new_length = calculate_path_length(new_order, swath_data, entry_point, exit_point)
            
            if new_length < best_length:
                best_order = new_order
                best_length = new_length
                order = new_order
        
        if iteration % 20 == 0 and iteration > 0:
            print(f"    Итерация {iteration}: длина = {best_length:.1f} м")
    
    print(f"    Оптимизация завершена. Лучшая длина: {best_length:.1f} м")
    
    # Преобразуем порядок в список индексов
    simple_order = [idx for idx, _ in best_order]
    
    return simple_order, swath_data

# ФУНКЦИЯ ДЛЯ РАСЧЕТА ДЛИНЫ ПУТИ
def calculate_path_length(order, swath_data, entry_point, exit_point):
    """Вычисляет общую длину пути для заданного порядка гонов."""
    if not order:
        return float('inf')
    
    total_length = 0
    current_point = entry_point
    
    for idx, from_start in order:
        swath = swath_data[idx]
        
        # Расстояние до гона
        if from_start:
            total_length += current_point.distance(swath['start'])
            current_point = swath['end']
        else:
            total_length += current_point.distance(swath['end'])
            current_point = swath['start']
        
        # Длина самого гона
        total_length += swath['length']
    
    # Расстояние до точки выезда
    total_length += current_point.distance(exit_point)
    
    return total_length

# ФУНКЦИЯ ПОСТРОЕНИЯ ПОЛНОЙ ТРАЕКТОРИИ
def build_complete_trajectory(order, swath_data, entry_point, exit_point, turn_radius, headland_polygon):
    """
    Строит полную траекторию с учетом гонов и разворотов.
    """
    print("  Построение полной траектории...")
    
    if not order or len(order) == 0:
        print("    Нет данных для построения траектории")
        return None, []
    
    trajectory_segments = []
    all_lines = []
    
    current_point = entry_point
    current_angle = None  # Текущее направление движения
    
    for i, (idx, from_start) in enumerate(order):
        swath = swath_data[idx]
        
        # Определяем параметры текущего гона
        if from_start:
            start_point = swath['start']
            end_point = swath['end']
            swath_direction = math.degrees(math.atan2(
                end_point.y - start_point.y,
                end_point.x - start_point.x
            ))
        else:
            start_point = swath['end']
            end_point = swath['start']
            swath_direction = math.degrees(math.atan2(
                start_point.y - end_point.y,
                start_point.x - end_point.x
            ))
        
        # 1. Переход от текущей точки к началу гона
        if not current_point.equals(start_point):
            if current_angle is not None:
                # Создаем плавный разворот
                turn_segments = create_smooth_turn(
                    current_point, current_angle,
                    start_point, swath_direction,
                    turn_radius, params['turn_type']
                )
                
                for segment in turn_segments:
                    trajectory_segments.append(('turn', segment))
                    all_lines.append(segment)
            else:
                # Первый переход - прямая линия
                transition = LineString([current_point, start_point])
                trajectory_segments.append(('transition', transition))
                all_lines.append(transition)
        
        # 2. Сам гон (рабочий ход)
        # Создаем линию гона в правильном направлении
        coords = list(swath['line'].coords)
        if not from_start:
            coords.reverse()
        
        working_line = LineString(coords)
        trajectory_segments.append(('working', working_line))
        all_lines.append(working_line)
        
        # Обновляем текущую позицию и направление
        current_point = end_point
        current_angle = swath_direction
    
    # 3. Переход к точке выезда
    if not current_point.equals(exit_point):
        if current_angle is not None:
            # Определяем направление к точке выезда
            exit_dx = exit_point.x - current_point.x
            exit_dy = exit_point.y - current_point.y
            exit_angle = math.degrees(math.atan2(exit_dy, exit_dx))
            
            # Создаем плавный разворот
            turn_segments = create_smooth_turn(
                current_point, current_angle,
                exit_point, exit_angle,
                turn_radius, params['turn_type']
            )
            
            for segment in turn_segments:
                trajectory_segments.append(('exit_turn', segment))
                all_lines.append(segment)
        else:
            # Прямой переход к точке выезда
            exit_transition = LineString([current_point, exit_point])
            trajectory_segments.append(('exit', exit_transition))
            all_lines.append(exit_transition)
    
    # Объединяем все сегменты в одну линию
    try:
        if all_lines:
            full_trajectory = linemerge(all_lines)
            
            # Если получилась мультилиния, пытаемся упростить
            if full_trajectory.geom_type == 'MultiLineString':
                # Объединяем все линии через буфер и обратное преобразование
                merged = unary_union(all_lines).buffer(0.1).boundary
                if merged.geom_type == 'LineString':
                    full_trajectory = merged
                else:
                    # Берем самую длинную линию
                    full_trajectory = max(full_trajectory.geoms, key=lambda g: g.length)
        else:
            full_trajectory = None
    except Exception as e:
        print(f"    Ошибка объединения траектории: {e}")
        full_trajectory = None
    
    # Расчет статистики
    if full_trajectory and not full_trajectory.is_empty:
        total_length = full_trajectory.length
        working_length = sum(seg['length'] for seg in swath_data)
        
        print(f"    Общая длина траектории: {total_length:.1f} м")
        print(f"    Длина рабочего хода: {working_length:.1f} м")
        print(f"    Эффективность: {working_length/total_length*100:.1f}%")
    
    return full_trajectory, trajectory_segments

# ФУНКЦИЯ СОХРАНЕНИЯ РЕЗУЛЬТАТОВ
def save_all_results(field_gdf, headland_zone, swaths, trajectory, output_paths):
    """Сохраняет все результаты в файлы GeoJSON."""
    print("  Сохранение результатов...")
    
    # 1. Сохраняем разворотную полосу
    if headland_zone and not headland_zone.is_empty:
        headland_gdf = gpd.GeoDataFrame(
            {'geometry': [headland_zone], 'type': ['headland']},
            crs=field_gdf.crs
        )
        headland_gdf.to_file(output_paths['headland'], driver='GeoJSON')
        print(f"    Разворотная полоса: {output_paths['headland']}")
    
    # 2. Сохраняем гоны
    if swaths:
        swaths_gdf = gpd.GeoDataFrame(
            {'geometry': swaths, 'type': ['swath'] * len(swaths)},
            crs=field_gdf.crs
        )
        swaths_gdf.to_file(output_paths['swaths'], driver='GeoJSON')
        print(f"    Гоны: {output_paths['swaths']}")
    
    # 3. Сохраняем траекторию
    if trajectory and not trajectory.is_empty:
        traj_gdf = gpd.GeoDataFrame(
            {'geometry': [trajectory], 
             'type': ['optimized_track'],
             'length_m': [trajectory.length]},
            crs=field_gdf.crs
        )
        traj_gdf.to_file(output_paths['track'], driver='GeoJSON')
        print(f"    Траектория: {output_paths['track']}")
    
    # 4. Создаем интерактивную карту
    try:
        center = field_gdf.geometry.unary_union.centroid
        m = folium.Map(location=[center.y, center.x], zoom_start=15)
        
        # Исходное поле
        folium.GeoJson(
            field_gdf,
            style_function=lambda x: {
                'color': 'green',
                'fillColor': 'lightgreen',
                'weight': 2,
                'fillOpacity': 0.2
            },
            name='Поле'
        ).add_to(m)
        
        # Разворотная полоса (если есть)
        if headland_zone and not headland_zone.is_empty:
            folium.GeoJson(
                headland_gdf,
                style_function=lambda x: {
                    'color': 'orange',
                    'fillColor': 'yellow',
                    'weight': 2,
                    'fillOpacity': 0.3
                },
                name='Разворотная полоса'
            ).add_to(m)
        
        # Гоны (если есть)
        if swaths:
            folium.GeoJson(
                swaths_gdf,
                style_function=lambda x: {
                    'color': 'blue',
                    'weight': 2,
                    'opacity': 0.7
                },
                name='Гоны'
            ).add_to(m)
        
        # Траектория
        if trajectory and not trajectory.is_empty:
            folium.GeoJson(
                traj_gdf,
                style_function=lambda x: {
                    'color': 'red',
                    'weight': 4,
                    'opacity': 0.9
                },
                name='Траектория'
            ).add_to(m)
        
        # Добавляем управление слоями
        folium.LayerControl().add_to(m)
        
        # Сохраняем карту
        m.save(output_paths['map'])
        print(f"    Карта: {output_paths['map']}")
        
    except Exception as e:
        print(f"    Ошибка создания карты: {e}")
    
    return True

# ОСНОВНАЯ ФУНКЦИЯ
def main():
    print("\n" + "="*60)
    print("ОПТИМИЗАЦИЯ ТРАЕКТОРИИ ОБРАБОТКИ ПОЛЯ")
    print("С УЧЕТОМ РАЗВОРОТНЫХ ПОЛОС И РАДИУСОВ ПОВОРОТА")
    print("="*60 + "\n")
    
    try:
        # 1. ЗАГРУЗКА ДАННЫХ
        print("1. ЗАГРУЗКА ДАННЫХ")
        field_gdf = gpd.read_file(params['field_path'], encoding=params['encoding'])
        entry_exit_gdf = gpd.read_file(params['entry_exit_path'], encoding=params['encoding'])
        
        # Проверяем и приводим к общей системе координат
        if not field_gdf.crs:
            field_gdf.crs = 'EPSG:3395'
        
        if not entry_exit_gdf.crs:
            entry_exit_gdf.crs = 'EPSG:3395'
        
        if str(field_gdf.crs) != 'EPSG:3395':
            field_gdf = field_gdf.to_crs('EPSG:3395')
        
        if str(entry_exit_gdf.crs) != 'EPSG:3395':
            entry_exit_gdf = entry_exit_gdf.to_crs('EPSG:3395')
        
        # Получаем полигон поля
        field_polygon = field_gdf.geometry.unary_union
        
        # Точки въезда/выезда
        if len(entry_exit_gdf) >= 2:
            entry_point = entry_exit_gdf.geometry.iloc[0]
            exit_point = entry_exit_gdf.geometry.iloc[1]
            print(f"    Используются заданные точки въезда/выезда")
        else:
            # Автоматическое определение точек на границе
            bounds = field_polygon.bounds
            entry_point = Point(bounds[0], bounds[1])
            exit_point = Point(bounds[2], bounds[3])
            print(f"    Автоматически определены точки въезда/выезда")
        
        area = field_polygon.area
        print(f"    Площадь поля: {area:.0f} м² ({area/10000:.2f} га)")
        
        # 2. СОЗДАНИЕ РАЗВОРОТНОЙ ПОЛОСЫ
        print("\n2. ПОДГОТОВКА РАЗВОРОТНОЙ ПОЛОСЫ")
        # Минимальная ширина = 2 * радиус поворота + запас
        min_headland = params['min_turn_radius'] * 2.5
        if params['headland_width'] < min_headland:
            params['headland_width'] = min_headland
            print(f"    Корректировка ширины разворотной полосы до {min_headland:.1f} м")
        
        work_polygon, headland_zone = create_headland_boundary(
            field_polygon, params['headland_width']
        )
        
        if work_polygon.is_empty:
            print("    Ошибка: не удалось создать рабочую зону")
            return
        
        # 3. ОПРЕДЕЛЕНИЕ НАПРАВЛЕНИЯ ОБРАБОТКИ
        print("\n3. ОПРЕДЕЛЕНИЕ ОПТИМАЛЬНОГО НАПРАВЛЕНИЯ")
        processing_direction = calculate_processing_direction_optimized(field_gdf)
        
        # 4. ГЕНЕРАЦИЯ ГОНОВ
        print("\n4. ГЕНЕРАЦИЯ ГОНОВ В РАБОЧЕЙ ЗОНЕ")
        swaths = generate_swaths_with_headland(
            work_polygon,
            processing_direction,
            params['working_width'],
            params['min_swath_length']
        )
        
        if not swaths:
            print("    Ошибка: не удалось сгенерировать гоны")
            return
        
        # 5. ОПТИМИЗАЦИЯ ПОРЯДКА ГОНОВ
        print("\n5. ОПТИМИЗАЦИЯ ПОРЯДКА ОБРАБОТКИ")
        # Подготавливаем данные для оптимизации
        swath_data = []
        for i, swath in enumerate(swaths):
            coords = list(swath.coords)
            start_point = Point(coords[0])
            end_point = Point(coords[-1])
            
            swath_data.append({
                'id': i,
                'line': swath,
                'start': start_point,
                'end': end_point,
                'length': swath.length,
                'center': swath.centroid
            })
        
        # Простой жадный алгоритм для начального порядка
        order = []
        current_point = entry_point
        visited = [False] * len(swath_data)
        
        while len(order) < len(swath_data):
            best_idx = -1
            best_dist = float('inf')
            best_from_start = True
            
            for i, swath in enumerate(swath_data):
                if visited[i]:
                    continue
                
                dist_to_start = current_point.distance(swath['start'])
                dist_to_end = current_point.distance(swath['end'])
                
                if dist_to_start < best_dist:
                    best_dist = dist_to_start
                    best_idx = i
                    best_from_start = True
                
                if dist_to_end < best_dist:
                    best_dist = dist_to_end
                    best_idx = i
                    best_from_start = False
            
            if best_idx == -1:
                break
            
            order.append((best_idx, best_from_start))
            visited[best_idx] = True
            
            if best_from_start:
                current_point = swath_data[best_idx]['end']
            else:
                current_point = swath_data[best_idx]['start']
        
        print(f"    Определен порядок обработки {len(order)} гонов")
        
        # 6. ПОСТРОЕНИЕ ТРАЕКТОРИИ
        print("\n6. ПОСТРОЕНИЕ ПОЛНОЙ ТРАЕКТОРИИ")
        trajectory, segments = build_complete_trajectory(
            order, swath_data, entry_point, exit_point,
            params['min_turn_radius'], work_polygon
        )
        
        if not trajectory or trajectory.is_empty:
            print("    Ошибка: не удалось построить траекторию")
            return
        
        # 7. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ
        print("\n7. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ")
        output_paths = {
            'headland': params['output_headland_path'],
            'swaths': params['output_swaths_path'],
            'track': params['output_track_path'],
            'map': params['output_map_path']
        }
        
        save_all_results(field_gdf, headland_zone, swaths, trajectory, output_paths)
        
        # 8. ИТОГИ
        print("\n" + "="*60)
        print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
        print("="*60)
        
        # Расчет статистик
        total_length = trajectory.length
        working_length = sum(swath.length for swath in swaths)
        efficiency = (working_length / total_length) * 100 if total_length > 0 else 0
        
        print(f"• Площадь поля: {area/10000:.2f} га")
        print(f"• Ширина разворотной полосы: {params['headland_width']:.1f} м")
        print(f"• Количество гонов: {len(swaths)}")
        print(f"• Общая длина траектории: {total_length:.1f} м")
        print(f"• Длина рабочего хода: {working_length:.1f} м")
        print(f"• Эффективность: {efficiency:.1f}%")
        print(f"• Направление обработки: {processing_direction:.1f}°")
        print(f"• Радиус поворота: {params['min_turn_radius']:.1f} м")
        
        # Файлы результатов
        print(f"\n• Файлы сохранены в: {Path(params['output_track_path']).parent}")
        print(f"  - Траектория: {Path(params['output_track_path']).name}")
        print(f"  - Гоны: {Path(params['output_swaths_path']).name}")
        print(f"  - Разворотная полоса: {Path(params['output_headland_path']).name}")
        print(f"  - Интерактивная карта: {Path(params['output_map_path']).name}")
        
        # Рекомендации
        print(f"\n• Рекомендации:")
        if efficiency < 70:
            print(f"  ⚠  Низкая эффективность. Попробуйте уменьшить ширину разворотной полосы.")
        if len(swaths) < 10:
            print(f"  ⚠  Мало гонов. Увеличьте ширину рабочей зоны.")
        
    except Exception as e:
        print(f"\nОшибка выполнения: {e}")
        import traceback
        traceback.print_exc()
    
    print("\n" + "="*60)
    print("ОБРАБОТКА ЗАВЕРШЕНА")
    print("="*60)

# ЗАПУСК ПРОГРАММЫ
if __name__ == "__main__":
    main()